In [1]:
require 'nn';
require 'paths'
local matio = require 'matio'
local tensor = matio.load('CIFAR2_reshaped.mat')
test_X = tensor.test_X

nRepeat = 4
labelLength = nRepeat * 2
numberTR0 = 5000
numberTR1 = 5000
numberTe0 = 500
numberTe1 = 500
label0 = torch.cat(torch.ones(nRepeat), torch.zeros(nRepeat),1)
label1 = torch.cat(torch.zeros(nRepeat), torch.ones(nRepeat),1)

train_Y = torch.cat(torch.repeatTensor(label0, numberTR0, 1), torch.repeatTensor(label1, numberTR1, 1), 1)
test_Y = torch.cat(torch.repeatTensor(label0, numberTe0, 1), torch.repeatTensor(label1, numberTe1, 1), 1)

tensors = {}
tensors.data = tensor.train_X
tensors.label = train_Y

--print(label0)
--print(train_Y[5001])
--print(torch.type(trainset))

In [2]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(tensors, 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
);
tensors.data = tensors.data:double() -- convert the data from a ByteTensor to a DoubleTensor.

function tensors:size() 
    return self.data:size(1) 
end

In [3]:
mean = {} -- store the mean, to normalize the test set in the future
stdv  = {} -- store the standard-deviation for the future
for i=1,3 do -- over each image channel
    mean[i] = tensors.data[{ {}, {i}, {}, {}  }]:mean() -- mean estimation
    print('Channel ' .. i .. ', Mean: ' .. mean[i])
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction, :add(value) add value to all elements in place.
    
    stdv[i] = tensors.data[{ {}, {i}, {}, {}  }]:std() -- std estimation
    print('Channel ' .. i .. ', Standard Deviation: ' .. stdv[i])
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling :div(value) divides all elements by value in place.
end

Channel 1, Mean: 122.45534121094	

Channel 1, Standard Deviation: 63.385616790565	


Channel 2, Mean: 120.61582177734	


Channel 2, Standard Deviation: 62.504468929637	


Channel 3, Mean: 111.08353691406	


Channel 3, Standard Deviation: 66.274687121541	


In [4]:
net = nn.Sequential()
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(3, 32, 5, 5)) -- 3 input image channels, 6 output channels, 5x5 convolution kernel
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity, think of this as sigmoid function 
net:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(32, 32, 5, 5)) --repeat convolution
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(32, 64, 5, 5)) --repeat convolution
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.View(64*4*4))                    -- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
net:add(nn.Linear(64*4*4, 120))             -- fully connected layer (matrix multiplication between input and weights), neural network, full connection.
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Dropout(0.5))
net:add(nn.Linear(84, labelLength))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
net:add(nn.Sigmoid()) 
--net:add(nn.LogSoftMax())                     -- converts the output to a log-probability. Useful for classification problems

In [5]:
criterion = nn.MSECriterion()
trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 5 -- just do 5 epochs of training.

In [ ]:
trainer:train(tensors)

# StochasticGradient: training	


In [ ]:
test_X = test_X:double()   -- convert from Byte tensor to Double tensor
for i=1,3 do -- over each image channel
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction    
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end

In [ ]:
correct = 0
for i=1,numberTe0 do
    local prediction = net:forward(test_X[i])
    if (prediction - label0)*(prediction - label0)< (prediction - label1) * (prediction -label1) then
        correct = correct + 1
    end
end

for i=numberTe0+1,numberTe0 + numberTe1 do
    local prediction = net:forward(test_X[i])
    if (prediction - label1)*(prediction - label1) < (prediction - label0)*(prediction - label0) then
        correct = correct + 1
    end
end

In [ ]:
print(correct, 100*correct/(numberTe0+numberTe1) .. ' % ')